In [2]:
import snowflake.connector
import csv
import pandas as pd
import numpy as np

In [3]:
ctx = snowflake.connector.connect(
    user='ssamichya',
    password='Lisnepal@2021',
    account='em35717.ap-south-1.aws',
    )

cursor = ctx.cursor()

In [3]:
data_sales = pd.read_csv("E:/sales_csv.csv")

In [4]:
data_sales

,ID,STORE_ID,PRODUCT_ID,CUSTOMER_ID,TRANSACTION_TIME,QUANTITY,AMOUNT,DISCOUNT
0,1,3,17,2.0,2021-11-11 13:39:14,12,28196.55,2819.66
1,2,4,15,5.0,2021-11-10 13:23:01,9,18034.61,901.73
2,3,2,11,5.0,2020-12-12 22:48:06,11,19433.22,971.66
3,4,2,1,3.0,2021-12-13 06:27:49,6,3623.98,181.20
4,5,4,4,NaN,2021-12-10 21:03:17,3,10707.11,535.36
...,...,...,...,...,...,...,...,...
95,96,3,1,6.0,2020-11-11 19:04:13,7,2337.76,116.89
96,97,2,17,3.0,2021-11-13 16:41:46,6,25751.48,2575.15
97,98,1,14,6.0,2020-12-11 17:02:26,5,13475.45,673.77
98,99,4,2,5.0,2020-12-11 00:11:49,4,17431.74,871.59


In [41]:
#write query and write the data to local
query_sales_aggregation = """with sales_aggregationCTE
AS(
	select *,YEAR(TRANSACTION_TIME) as Year,MONTH(TRANSACTION_TIME) as Month from BHATBHATENI_KEC_DWH_SURAJ.STG.SalesSTG
)
select Year, Month, CONCAT(Year,'/',MONTHNAME(TRANSACTION_TIME)) as Year_Month, sum(Amount) as Total_Sales from sales_aggregationCTE
GROUP BY Year_Month, Year, Month
ORDER BY Year,Month;"""
result = cursor.execute(query_sales_aggregation).fetchall()
header = ['Year','Month','Year_Month', 'Total_Sales']
writer = csv.writer(open('E:/sales_aggregate.csv','w', newline = ''))
writer.writerow(header)
for item in result:
    writer.writerow(item)

In [42]:
cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.TMP")
cursor.execute("create or replace table Sales_agg_TMP (Year varchar(50), Month varchar(50), Year_Month varchar(50), Total_Sales Number)")

In [43]:
#load the aggregate sales to the SalesTMP file
data_sales_aggregate = pd.read_csv('E:/sales_aggregate.csv')
for index, row in data_sales_aggregate.iterrows():
    cursor.execute("insert into Sales_agg_TMP values(%s,%s,%s,%s)",(row.Year, row.Month,row.Year_Month, row.Total_Sales))

In [44]:
#add maintainence column to the TMP file and then load it to the SalesTGT table under the TGT schema in DWH
cursor.execute("use BHATBHATENI_KEC_DWH_SURAJ.TGT")
cursor.execute("create or replace table Sales_agg_TGT (ID int, Year varchar(50), Month varchar(50), Year_Month varchar(50), Total_Sales Number, Rcd_ins_ts TIMESTAMP_LTZ(9), Rcd_upd_ts TIMESTAMP_LTZ(9))")

In [45]:
import datetime
ts = datetime.datetime.now()

In [17]:
data_sales_aggregate

,Year,Month,Total_Sales
0,2020,9,78507
1,2020,10,143385
2,2020,11,168810
3,2020,12,226882
4,2021,9,124750
5,2021,10,187908
6,2021,11,269806
7,2021,12,353207


In [46]:
for index,row in data_sales_aggregate.iterrows():
    cursor.execute("insert into BHATBHATENI_KEC_DWH_SURAJ.TGT.Sales_agg_TGT values(%s, %s, %s, %s, %s, %s, %s)",(index+1, row.Year,row.Month, row.Year_Month, row.Total_Sales,ts,ts))

In [51]:
cursor.execute("select * from BHATBHATENI_KEC_DWH_SURAJ.TGT.Sales_agg_TGT")
result = cursor.fetchall()
header = ['ID','Year','Month','Year_Month', 'Total_Sales','Rcd_ins_ts', 'Rcd_upd_ts']
writer = csv.writer(open('E:/final_sales_aggregate.csv','w', newline = ''))
writer.writerow(header)
for item in result:
    writer.writerow(item)

In [52]:
data_final_salesSTG = pd.read_csv('E:/final_sales_aggregate.csv')

In [54]:
data_final_salesSTG

,ID,Year,Month,Year_Month,Total_Sales,Rcd_ins_ts,Rcd_upd_ts
0,1,2020,9,2020/Sep,78507,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
1,2,2020,10,2020/Oct,143385,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
2,3,2020,11,2020/Nov,168810,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
3,4,2020,12,2020/Dec,226882,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
4,5,2021,9,2021/Sep,124750,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
5,6,2021,10,2021/Oct,187908,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
6,7,2021,11,2021/Nov,269806,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
7,8,2021,12,2021/Dec,353207,2021-12-31 14:23:04.392266-08:00,2021-12-31 14:23:04.392266-08:00
